In [1]:
import pandas as pd
import os

# --- 設定項目 ---
# 論文リストが記録されたCSVファイル
RESULTS_CSV_PATH = '../data/processed/citing_papers_with_paths.csv'
# 出力先のディレクトリとファイル名
OUTPUT_DIR = '../data/ground_truth'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'annotation_target_list.csv')
# 抽出したいサンプル数
SAMPLE_SIZE = 200

# --- 1. データの読み込みとフィルタリング ---
try:
    df_results = pd.read_csv(RESULTS_CSV_PATH)
    
    # 本文XMLのダウンロードに成功した（cached含む）論文のみをサンプリングの母集団とする
    df_success = df_results[
        df_results['download_status'].str.startswith('success', na=False)
    ].copy()
    
    print(f"XML取得に成功した {len(df_success)} 件の論文からサンプリングを行います。")

except FileNotFoundError:
    print(f"エラー: '{RESULTS_CSV_PATH}' が見つかりません。")
    df_success = pd.DataFrame()

# --- 2. 単純ランダムサンプリングの実行 ---
if not df_success.empty and len(df_success) >= SAMPLE_SIZE:
    # random_stateを指定することで、いつでも同じランダム結果を再現できます
    sample_df = df_success.sample(n=SAMPLE_SIZE, random_state=42)
    
    # アノテーション作業で必要な列に絞って保存すると便利です
    columns_to_save = [
        'citing_paper_eid', 
        'citing_paper_doi', 
        'citing_paper_title',
        'cited_data_paper_title' # どのデータ論文を引用しているかの確認用
    ]
    
    # 出力先ディレクトリが存在しない場合は作成
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    # サンプリング結果をCSVファイルとして保存
    sample_df[columns_to_save].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    
    print(f"\nサンプリング完了。")
    print(f"アノテーション対象 {SAMPLE_SIZE} 件のリストを '{OUTPUT_FILE}' に保存しました。")
    
    print("\n--- サンプルリストの先頭5件 ---")
    print(sample_df[columns_to_save].head())
else:
    print(f"サンプリング対象となる、ダウンロードに成功した論文が{SAMPLE_SIZE}件未満です。")

XML取得に成功した 40720 件の論文からサンプリングを行います。

サンプリング完了。
アノテーション対象 200 件のリストを '../data/ground_truth\annotation_target_list.csv' に保存しました。

--- サンプルリストの先頭5件 ---
          citing_paper_eid                 citing_paper_doi  \
81370   2-s2.0-85211640167  10.1016/j.marpolbul.2024.117442   
10064   2-s2.0-85210281314   10.1016/j.jaridenv.2024.105282   
39929   2-s2.0-85171680307   10.1016/j.revpalbo.2023.104989   
101260  2-s2.0-85142708771     10.1016/j.foreco.2022.120653   
68305   2-s2.0-85194770743    10.1016/j.jnucmat.2024.155194   

                                       citing_paper_title  \
81370   Multi-indicator assessment of heavy metal poll...   
10064   Potential effects of climate change on cacti d...   
39929   Approaches to pollen taxonomic harmonisation i...   
101260  Allometric equations to estimate the dry mass ...   
68305   Microstructural evolution in doped high entrop...   

                                   cited_data_paper_title  
81370   Pollution load index for heavy metals